# WSDM - KKBox's Music Recommendation Challenge

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load the preprocessed data
- train: training dataset
- test: test dataset
- items: songs
- items_extra: extra songs information
- users: members

In [13]:
# Use pandas to load the zip csv file
df    = pd.read_csv('./process/train.csv.gz', compression="gzip")
test  = pd.read_csv('./process/test.csv.gz', compression="gzip")
songs = pd.read_csv('./process/songs.csv.gz', compression="gzip")
songs_extra = pd.read_csv('./process/song_extra_info.csv.gz', compression="gzip")
users = pd.read_csv('./process/members.csv.gz', compression="gzip")